In [1]:
import tensorflow as tf

In [2]:
class Conv(tf.keras.layers.Layer):
    def __init__(self,
                 filters:int,
                 kernel_size,
                 strides = (1,1),
                 padding = 'same'
                ):
        super(Conv, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        
        self.C = tf.keras.layers.Conv2D(filters = self.filters,
                                        kernel_size = self.kernel_size,
                                        strides = self.strides,
                                        padding = self.padding,
                                        use_bias = False
                                       )
        self.BN = tf.keras.layers.BatchNormalization()
        
    def call(self, X):
        y = self.C(X)
        y = self.BN(y)
        y = tf.nn.relu(y)
        return y

In [3]:
class InceptionA(tf.keras.layers.Layer):
    def __init__(self):
        super(InceptionA, self).__init__()
        
        self.path_1 = tf.keras.Sequential([
            Conv(filters = 64,
                 kernel_size = (1,1),
                 padding = 'valid'
                ),
            Conv(filters = 96,
                 kernel_size = (3,3)
                ),
            Conv(filters = 96,
                 kernel_size = (3,3)
                )
        ])
        self.path_2 = tf.keras.Sequential([
            Conv(filters = 48,
                 kernel_size = (1,1),
                 padding = 'valid'
                ),
            Conv(filters = 64,
                 kernel_size = (3,3)
                ),
        ])
        self.path_3 = tf.keras.Sequential([
            tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                      strides = (1,1),
                                      padding = 'same'
                                     ),
            Conv(filters = 64,
                 kernel_size = (1,1),
                 padding = 'valid'
                )
        ])
        self.path_4 = tf.keras.Sequential([
            Conv(filters = 64,
                 kernel_size = (1,1),
                 padding = 'valid'
                )
        ])
        
    def call(self, X):
        y1 = self.path_1(X)
        y2 = self.path_2(X)
        y3 = self.path_3(X)
        y4 = self.path_4(X)
        return tf.concat([y1,y2,y3,y4], axis = -1)
        
        
class InceptionB(tf.keras.layers.Layer):
    def __init__(self, hidden_channels:int):
        super(InceptionB, self).__init__()
        self.hidden_channels = hidden_channels
        
        self.path_1 = tf.keras.Sequential([
            Conv(filters = self.hidden_channels,
                 kernel_size = (1,1),
                 padding = 'valid'
                ),
            Conv(filters = self.hidden_channels,
                 kernel_size = (1,7),
                ),
            Conv(filters = self.hidden_channels,
                 kernel_size = (7,1)
                ),
            Conv(filters = self.hidden_channels,
                 kernel_size = (1,7)
                ),
            Conv(filters = 192,
                 kernel_size = (7,1)
                )
        ])
        self.path_2 = tf.keras.Sequential([
            Conv(filters = self.hidden_channels,
                 kernel_size = (1,1),
                 padding = 'valid'
                ),
            Conv(filters = self.hidden_channels,
                 kernel_size = (1,7),
                ),
            Conv(filters = 192,
                 kernel_size = (7,1),
                )
        ])
        self.path_3 = tf.keras.Sequential([
            tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                      strides = (1,1),
                                      padding = 'same'
                                     ),
            Conv(filters = 192,
                 kernel_size = (1,1),
                 padding = 'valid'
                )
        ])
        self.path_4 = tf.keras.Sequential([
            Conv(filters = 192,
                 kernel_size = (1,1),
                 padding = 'valid'
                )
        ])
        
    def call(self, X):
        y1 = self.path_1(X)
        y2 = self.path_2(X)
        y3 = self.path_3(X)
        y4 = self.path_4(X)
        return tf.concat([y1,y2,y3,y4], axis = -1)
    
class InceptionC(tf.keras.layers.Layer):
    def __init__(self):
        super(InceptionC, self).__init__()
        
        self.path_1 = tf.keras.Sequential([ 
            Conv(filters = 448,
                 kernel_size = (1,1),
                 padding = 'valid'
                ),
            Conv(filters = 384,
                 kernel_size = (3,3)
                )
        ])
        self.path_1_1 = Conv(filters = 384,
                             kernel_size = (1,3)
                            )
        self.path_1_2 = Conv(filters = 384,
                             kernel_size = (3,1)
                            )
        self.path_2 = Conv(filters = 384,
                           kernel_size = (1,1),
                           padding = 'valid'
                          )
        self.path_2_1 = Conv(filters = 384,
                             kernel_size = (1,3)
                            )
        self.path_2_2 = Conv(filters = 384,
                             kernel_size = (3,1)
                            )
        self.path_3 = tf.keras.Sequential([
            tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                      strides = (1,1),
                                      padding = 'same'
                                     ),
            Conv(filters = 192,
                 kernel_size = (1,1),
                 padding = 'valid'
                )
        ])
        self.path_4 = Conv(filters = 320,
                           kernel_size = (1,1),
                           padding = 'valid'
                          )
    
    def call(self, X):
        p1 = self.path_1(X)
        y1 = tf.concat([self.path_1_1(p1), self.path_1_2(p1)], axis = -1)
        p2 = self.path_2(X)
        y2 = tf.concat([self.path_2_1(p2), self.path_2_2(p2)], axis = -1)
        y3 = self.path_3(X)
        y4 = self.path_4(X)
        return tf.concat([y1,y2,y3,y4], axis = -1)

    
class Downsampling(tf.keras.layers.Layer):
    def __init__(self, hidden_channels:int, add_channels:int=0):
        super(Downsampling, self).__init__()
        self.hidden_channels = hidden_channels
        self.add_channels = add_channels
        
        self.path_1 = tf.keras.Sequential([
            Conv(filters = self.hidden_channels,
                 kernel_size = (1,1),
                 padding = 'valid'
                ),
            Conv(filters = 178 + self.add_channels,
                 kernel_size = (3,3)
                ),
            Conv(filters = 178 + self.add_channels,
                 kernel_size = (3,3),
                 strides = (2,2),
                 padding = 'valid'
                )
        ])
        self.path_2 = tf.keras.Sequential([
            Conv(filters = self.hidden_channels,
                 kernel_size = (1,1),
                 padding = 'valid'
                ),
            Conv(filters = 302 + self.add_channels,
                 kernel_size = (3,3),
                 strides = (2,2),
                 padding = 'valid'
                )
        ])
        self.path_3 = tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                                strides = (2,2),
                                                padding = 'valid'
                                               )
        
    def call(self, X):
        y1 = self.path_1(X)
        y2 = self.path_2(X)
        y3 = self.path_3(X)
        return tf.concat([y1,y2,y3], axis = -1)

In [4]:
class InceptionV3(tf.keras.models.Model):
    def __init__(self, n_labels:int, last_activation:str = 'softmax'):
        super(InceptionV3, self).__init__()
        self.n_labels = n_labels
        self.last_activation = last_activation
        
        self.Stem = tf.keras.Sequential([
            Conv(filters = 32,
                 kernel_size = (3,3),
                 strides = (2,2),
                 padding = 'valid'
                ),
            Conv(filters = 32,
                 kernel_size = (3,3),
                 padding = 'valid'
                ),
            Conv(filters = 64,
                 kernel_size = (3,3)
                ),
            tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                      strides = (2,2),
                                      padding = 'valid'
                                     ),
            Conv(filters = 80,
                 kernel_size = (3,3),
                 padding = 'valid'
                ),
            Conv(filters = 192,
                 kernel_size = (3,3),
                 strides = (2,2),
                 padding = 'valid'
                ),
            Conv(filters = 288,
                 kernel_size = (3,3)
                )
        ])
        
        self.InceptionBlocksA = tf.keras.Sequential([
            InceptionA() for _ in range(3)
        ])
        self.D1 = Downsampling(64)
        
        self.InceptionBlocksB = tf.keras.Sequential([
            InceptionB(128),
            InceptionB(160),
            InceptionB(160),
            InceptionB(160),
            InceptionB(192)
        ])
        self.AuxClassifier = tf.keras.Sequential([
            tf.keras.layers.AveragePooling2D(pool_size = (5,5),
                                             strides = (3,3),
                                             padding = 'valid'
                                            ),
            Conv(filters = 128,
                 kernel_size = (1,1),
                 padding = 'valid'
                ),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1024, activation = 'relu'),
            tf.keras.layers.Dense(self.n_labels, self.last_activation)
        ]) 
        self.D2 = Downsampling(192, 16)
        
        self.InceptionBlocksC = tf.keras.Sequential([
            InceptionC(),
            InceptionC(),
            InceptionC()
        ])
        
        self.Classifier = tf.keras.Sequential([
            tf.keras.layers.AveragePooling2D(pool_size = (7,7),
                                             padding = 'valid'
                                            ),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(self.n_labels, activation = self.last_activation)
        ])
    
    def call(self, X):
        y = self.Stem(X)
        
        y = self.InceptionBlocksA(y)
                
        y = self.D1(y)

        y = self.InceptionBlocksB(y)
        y_aux = self.AuxClassifier(y)
        
        y = self.D2(y)
        
        y = self.InceptionBlocksC(y)
        
        y = self.Classifier(y)
        
        return y, y_aux

In [5]:
inceptionv3 = InceptionV3(1000)

In [6]:
inceptionv3.build([16,299,299,3])

In [7]:
inceptionv3.summary()

Model: "inception_v3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (16, 35, 35, 288)         713248    
_________________________________________________________________
sequential_13 (Sequential)   (16, 35, 35, 288)         710976    
_________________________________________________________________
downsampling (Downsampling)  multiple                  601644    
_________________________________________________________________
sequential_36 (Sequential)   (16, 17, 17, 768)         8529664   
_________________________________________________________________
sequential_37 (Sequential)   (16, 1000)                4401640   
_________________________________________________________________
downsampling_1 (Downsampling multiple                  1522732   
_________________________________________________________________
sequential_46 (Sequential)   (16, 8, 8, 2048)         